recurrence relation example
https://jakevdp.github.io/blog/2013/06/15/numba-vs-cython-take-2/

In [75]:
from numba import jit

np.random.seed(1234)
pd.set_option('max_row',12)
s = Series(np.random.randn(1e5))
com = 0.5
s

0        0.471435
1       -1.190976
2        1.432707
3       -0.312652
4       -0.720589
5        0.887163
           ...   
99994   -0.940113
99995   -1.478211
99996    0.279401
99997    0.029286
99998   -1.220531
99999    0.384112
dtype: float64

In [124]:
def python(s):
    output = Series(index=range(len(s)))

    alpha = 1. / (1. + com)
    old_weight = 1.0
    new_weight = 1.0
    weighted_avg = s[0]
    output[0] = weighted_avg
    
    for i in xrange(1,len(s)):
        v = s[i]
        old_weight *= (1-alpha)
        weighted_avg = ((old_weight * weighted_avg) + (new_weight * v)) / (old_weight + new_weight)
        old_weight += new_weight
        output[i] = weighted_avg
        
    return output

def cython(s):
    return pd.ewma(s,com=com,adjust=True)

def numba(s):
 
    @jit
    def f(arr, output):
        alpha = 1. / (1. + com)
        old_weight = 1.0
        new_weight = 1.0
        weighted_avg = arr[0]
        output[0] = weighted_avg
    
        for i in range(1,arr.shape[0]):
            v = arr[i]
            old_weight *= (1-alpha)
            weighted_avg = ((old_weight * weighted_avg) + (new_weight * v)) / (old_weight + new_weight)
            old_weight += new_weight
            output[i] = weighted_avg
    
    output = np.empty(len(s),dtype='float64')
    f(s.values, output)
    return Series(output)


In [120]:
result1 = python(s)
result2 = cython(s)
result3 = numba(s)
result1.equals(result2) and result1.equals(result3)

True

In [121]:
%timeit python(s)

1 loops, best of 3: 1.09 s per loop


In [122]:
%timeit cython(s)

100 loops, best of 3: 3.88 ms per loop


In [123]:
%timeit numba(s)

10 loops, best of 3: 53.9 ms per loop
